In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
import torch

from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import LambdaLR

# Get the directory containing the notebook
notebook_dir = os.path.dirname(os.path.abspath("__file__"))

# Add the directory containing the notebook to sys.path
sys.path.append(notebook_dir)

# Add the parent directory (which contains the 'dataloaders' directory) to sys.path
parent_dir = os.path.abspath(os.path.join(notebook_dir, '.'))
sys.path.append(parent_dir)


In [ ]:
from functions.loader import getLoader
from functions.display_things import *
from functions.trainFuncs import *
from functions.STGCN import *

In [ ]:
station = "varberg"
future_steps = 36
seq_len = 576
batch_size = 64
random_seed = 42

epochs = 60
warmup_steps = int(epochs * 0.2)
learning_rate = 0.01
hyperN_start=0


# Data Prep

In [ ]:
train_loader, val_loader, test_loader = getLoader(station=station, future_steps=future_steps,
                                                  seq_len=seq_len, batch_size=batch_size,
                                                  random_seed=random_seed)

# Model Making

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from torch_geometric.nn import GCNConv

edge_index_single = torch.tensor([[i, j] for i in range(5) for j in range(5) if i != j], dtype=torch.long).t()
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(nn.Module):
    def __init__(self, in_channels=1, gcn_hidden_channels=8, gcn_layers=1):
        super(GCN, self).__init__()
        self.in_conv = GCNConv(in_channels, gcn_hidden_channels)
        self.hidden_convs = nn.ModuleList(
            [GCNConv(gcn_hidden_channels, gcn_hidden_channels) for _ in range(gcn_layers - 1)]
        )

    def forward(self, x, edge_index, edge_attr, weights, batch):        
        x = x.float()
        
        x = self.in_conv(x, edge_index)
        for i, conv in enumerate(self.hidden_convs[:-1]):
            x = F.relu(x)
            x = conv(x, edge_index)
        x = F.relu(x)

        if weights:
            x = reshape_to_batches(x, batch)
            print(x.shape)  # [64, 5, 576, 4]
            batch_size, num_stations, seq_len, feature_size = x.shape
            gcn_last_layer_weight, gcn_last_layer_bias = weights[0]

            # Reshape to apply linear transformation
            x = x.view(batch_size * num_stations, seq_len, feature_size)
            x = x.view(-1, feature_size)
            x = F.linear(x, gcn_last_layer_weight, gcn_last_layer_bias)
            x = x.view(batch_size * num_stations, seq_len, -1)
            x = x.view(batch_size, num_stations, seq_len, -1)
            x = reshape_from_batches(x)
        return x
class SimpleTransformer(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size, nhead, seq_length, num_layers=1, dropout=0.1):
        super(SimpleTransformer, self).__init__()

        self.seq_length = seq_length
        self.output_size = output_size
        self.hidden_layer_size = hidden_layer_size
        
        self.embeddingIn = nn.Linear(input_size, hidden_layer_size)
        self.embeddingTGT = nn.Linear(output_size, hidden_layer_size)
        
        self.PositionalEncoding = PositionalEncoding(max_len=1000, d_model=hidden_layer_size)
        
        self.encoder_layers = nn.ModuleList([
            nn.TransformerEncoderLayer(d_model=hidden_layer_size, nhead=nhead, 
                                       dim_feedforward=4*hidden_layer_size, dropout=dropout, 
                                       activation='gelu')
            for _ in range(num_layers)
        ])
        
        self.decoder_layers = nn.ModuleList([
            nn.TransformerDecoderLayer(d_model=hidden_layer_size, nhead=nhead,
                                       dim_feedforward=4*hidden_layer_size, dropout=dropout, 
                                       activation='gelu')
            for _ in range(num_layers)
        ])

        self.linear1 = nn.Linear(hidden_layer_size, output_size)
                
    def generate_square_subsequent_mask(self, sz):
        mask = torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)
        return mask
        
    def forward(self, x, tgt=None, last_value=None, inference=False, weights=None):
        last_value = torch.unsqueeze(last_value, dim=2)

        initial_tgt = last_value
        
        tgt_input = torch.cat([last_value, tgt[:, :-1]], dim=1)
        
        x = self.embeddingIn(x)
        x = self.PositionalEncoding(x)
        enc_mask = self.generate_square_subsequent_mask(x.size(1)).to(tgt.device)
        x = x.permute(1, 0, 2)
        
        for layer in self.encoder_layers[:-1]:
            x = layer(x, src_mask=enc_mask)

        if weights:
            batch_size, num_stations, seq_len, feature_size = x.shape
            transformer_last_layer_weight, transformer_last_layer_bias = weights[1]

            # Reshape to apply linear transformation
            x = x.view(batch_size * num_stations, seq_len, feature_size)
            x = x.view(-1, feature_size)
            x = F.linear(x, transformer_last_layer_weight, transformer_last_layer_bias)
            x = x.view(batch_size * num_stations, seq_len, -1)
            x = x.view(batch_size, num_stations, seq_len, -1)
            x = x.permute(1, 0, 2)
        else:
            x = self.encoder_layers[-1](x, src_mask=enc_mask)
        
        encoder_output = x.permute(1, 0, 2)
                
        if inference:
            tgt_gen = initial_tgt
            generated_sequence = torch.zeros((initial_tgt.size(0), self.seq_length, self.output_size), device=x.device)
            encoder_output = encoder_output.permute(1, 0, 2)

            for i in range(self.seq_length):
                tgt_emb = self.embeddingTGT(tgt_gen)
                tgt_emb = self.PositionalEncoding(tgt_emb)
                tgt_emb = tgt_emb.permute(1, 0, 2)

                for layer in self.decoder_layers[:-1]:
                    decoder_output = layer(tgt_emb, encoder_output)
                
                output_step = self.linear1(decoder_output[-1, :, :])
                output_step = output_step.unsqueeze(1) 

                generated_sequence[:, i:i+1, :] = output_step

                tgt_gen = torch.cat((tgt_gen, output_step), dim=1)

                if tgt_gen.size(1) > self.seq_length:
                    tgt_gen = tgt_gen[:, 1:, :]

            return generated_sequence

        else:
            tgt = self.embeddingTGT(tgt_input)
            tgt = self.PositionalEncoding(tgt)
            tgt = tgt.permute(1, 0, 2)

            tgt_mask = self.generate_square_subsequent_mask(tgt.size(0)).to(tgt.device)

            encoder_output = encoder_output.permute(1, 0, 2)
            
            for layer in self.decoder_layers[:-1]:
                decoder_output = layer(tgt, encoder_output, tgt_mask=tgt_mask)
            
            output = self.linear1(decoder_output)

            return output.permute(1, 0, 2)



class STGCN(nn.Module):
    def __init__(self, in_channels, gcn_layers, hidden_channels, transformer_hidden_size, transformer_num_layers, transformer_nhead, out_channels):
        super(STGCN, self).__init__()
        self.GCN = GCN(in_channels=in_channels, gcn_hidden_channels=hidden_channels, gcn_layers=gcn_layers)

        self.transformer = SimpleTransformer(
            input_size=hidden_channels, hidden_layer_size=transformer_hidden_size,
            output_size=out_channels, seq_length=36, num_layers=transformer_num_layers,
            nhead=transformer_nhead
        ).cuda()

    def forward(self, data, inference=False, weights=None):
        batch = data.batch
        label = data.y
        label = torch.squeeze(label, 2)

        data.x = data.x.float()
        data.edge_attr = data.edge_attr.float()
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr

        x = self.GCN(x, edge_index, edge_attr, weights=weights, batch=batch)
        x = reshape_to_batches(x, batch)
        last_value = reshape_to_batches(data.x[:, -1, :], batch)
        label = reshape_to_batches(label, batch)

        predictions = []

        for station_data, station_label, station_last_value in zip(x.permute(1, 0, 2, 3), label.permute(1, 0, 2, 3), last_value.permute(1, 0, 2)):
            output = self.transformer(station_data, station_label, station_last_value, inference, weights=weights)
            predictions.append(output)

        predictions = torch.stack(predictions, dim=1)
        return predictions


class WeightGenerator(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(WeightGenerator, self).__init__()
        self.fc1 = nn.Linear(in_channels, hidden_channels)
        self.fc2 = nn.Linear(hidden_channels, out_channels)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class FusionNetwork(nn.Module):
    def __init__(self, stgcn_params, weight_gen_params):
        super(FusionNetwork, self).__init__()
        self.stgcn = STGCN(**stgcn_params).cuda()
        self.weight_generator = WeightGenerator(**weight_gen_params).cuda()
        self.hyper_on_state = False

    def hyper_on(self, state):
        self.hyper_on_state = state

    def freeze_stgcn(self):
        for param in self.stgcn.parameters():
            param.requires_grad = False

    def unfreeze_stgcn(self):
        for param in self.stgcn.parameters():
            param.requires_grad = True

    def forward(self, data, inference=False):
        if self.hyper_on_state:
            x_flat = data.x.view(data.x.size(0), -1)
            generated_weights = self.weight_generator(x_flat)

            gcn_last_layer_weight = generated_weights[:, :self.stgcn.GCN.hidden_convs[-1].lin.weight.numel()]
            gcn_last_layer_weight = gcn_last_layer_weight.reshape(
                (gcn_last_layer_weight.shape[0] // 5, 5, 4, 4)
            )
            gcn_last_layer_weight = gcn_last_layer_weight.sum(dim=1)

            transformer_last_layer_weight = generated_weights[:, self.stgcn.GCN.hidden_convs[-1].lin.weight.numel():]
            transformer_last_layer_weight = transformer_last_layer_weight.reshape(
                (transformer_last_layer_weight.shape[0] // 5, 5, 36, 12)
            )
            transformer_last_layer_weight = transformer_last_layer_weight.sum(dim=1)

            # Ensure the generated weights have requires_grad=True
            gcn_last_layer_weight = gcn_last_layer_weight.detach().clone().requires_grad_(True)
            transformer_last_layer_weight = transformer_last_layer_weight.detach().clone().requires_grad_(True)

            gcn_last_layer_bias = torch.zeros(gcn_last_layer_weight.size(0), device=gcn_last_layer_weight.device).requires_grad_(True)
            transformer_last_layer_bias = torch.zeros(transformer_last_layer_weight.size(0), device=transformer_last_layer_weight.device).requires_grad_(True)

            weights = [(gcn_last_layer_weight, gcn_last_layer_bias), (transformer_last_layer_weight, transformer_last_layer_bias)]

            predictions = self.stgcn(data, inference, weights=weights)
        else:
            predictions = self.stgcn(data, inference)

        return predictions



# Training

In [ ]:

# Example usage:
stgcn_params = {
    'in_channels': 1,
    'gcn_layers': 3,
    'hidden_channels': 4,
    'transformer_hidden_size': 12,
    'transformer_num_layers': 2,
    'transformer_nhead': 2,
    'out_channels': 1
}

weight_gen_params = {
    'in_channels': 1 * 576,  # Adjust based on input dimensions
    'hidden_channels': 64,
    'out_channels': 4 * 4 + 2 * 12 * 12 + 150 - 6 # Adjust based on the weight dimensions
}

model = FusionNetwork(stgcn_params, weight_gen_params).cuda()

model.load_state_dict(torch.load('HyperNetwork_pretrained_on_varnamo.pth'))

model.freeze_stgcn()


In [ ]:

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()    

# Define the lambda function for scheduling with Noam-style learning rate decay
def lr_lambda(current_step: int, d_model: int, warmup_steps: int) -> float:
    current_step+=1
    return (d_model ** (-0.5)) * min((current_step ** (-0.5)), current_step * (warmup_steps ** (-1.5)))

d_model = stgcn_params['transformer_hidden_size']
scheduler = LambdaLR(optimizer, lr_lambda=lambda step: lr_lambda(step, d_model, warmup_steps))    

# Now pass the scheduler to the training function
best_model, best_epoch, train_losses, val_losses, lrs = a_proper_training(
    epochs, model, optimizer, criterion, train_loader, val_loader, scheduler, hyperN_start=hyperN_start
)

torch.save(best_model.state_dict(), "HyperNetwork_pretrained_on_varnamo-finetuned_on_varberg.pth")

plt.plot(train_losses, label="train")
plt.plot(val_losses, label="val")
#plt.plot(lrs, label="learning rates")

plt.title("MSE Loss")
plt.legend()


In [ ]:
best_model.eval()

predictAndDisplay(station, val_loader, best_model)